# Mixture of Experts (MoE) 简介

## 1. 基本思想

- **普通 FFN**：在 Transformer 中，前馈层是一个固定的全连接网络：
  $$
  x \to W_1 \to \sigma \to W_2 \to y
  $$

- **MoE**：引入多个专家（experts），每个专家都是一个 FFN。输入 token 不再经过所有专家，而是由路由器 (router) 选择其中的少数几个（如 Top-1 或 Top-2）来处理，再将结果加权合并。

- **优势**：在不增加推理 FLOPs 的前提下，大幅增加模型参数量，实现稀疏激活。

---

## 2. 数学公式

### (1) 路由概率
对输入向量 $x \in \mathbb{R}^d$，路由器计算专家得分：
$$
g(x) = \text{softmax}(W_r x) \in \mathbb{R}^E
$$
其中 $E$ 为专家数，$W_r \in \mathbb{R}^{E \times d}$，$g_i(x)$ 表示专家 $i$ 被选中的概率。

### (2) Top-k 路由
选择前 $k$ 个专家：
$$
\text{TopK}(g(x)) = \{(i_1, p_1), (i_2, p_2), \dots, (i_k, p_k)\}
$$
其中 $i_j$ 是专家索引，$p_j$ 是归一化后的概率。

### (3) 专家计算
每个专家是一个前馈网络：
$$
f_i(x) = W_{2,i} \cdot \sigma(W_{1,i} x + b_{1,i}) + b_{2,i}
$$

### (4) MoE 输出
MoE 的最终输出是加权和：
$$
y = \sum_{j=1}^k p_j \cdot f_{i_j}(x)
$$

---

## 3. 正则化损失

### (1) 负载均衡损失
防止路由器过度集中在少数专家：
$$
L_{\text{aux}} = E \cdot \sum_{i=1}^E \big( \text{importance}_i \cdot \text{load}_i \big)
$$
其中  
- $\text{importance}_i = \sum_x g_i(x)$  
- $\text{load}_i = \sum_x \mathbf{1}[\arg\max g(x) = i]$

### (2) Router z-loss
约束 router logits 的数值范围：
$$
L_z = \mathbb{E}\Bigg[\Big(\log \sum_{i=1}^E \exp((W_r x)_i)\Big)^2\Bigg]
$$

---

## 4. 直观理解

- 普通 FFN：所有 token 走同一个专家。  
- MoE：每个 token 根据路由结果走不同专家，像医院分诊。  
- 好处：参数量可以无限增大，但单 token 的计算开销保持可控。

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Expert：普通 FFN
class Expert(nn.Module):
    def __init__(self, d_model, d_hidden):
        super().__init__()
        self.fc1 = nn.Linear(d_model, d_hidden)
        self.fc2 = nn.Linear(d_hidden, d_model)

    def forward(self, x):
        return self.fc2(F.gelu(self.fc1(x)))

# Router：Top-2 路由
class Router(nn.Module):
    def __init__(self, d_model, n_experts):
        super().__init__()
        self.gate = nn.Linear(d_model, n_experts, bias=False)

    def forward(self, x):
        logits = self.gate(x)                      # [B,T,E]
        probs = F.softmax(logits, dim=-1)
        topk_prob, topk_idx = probs.topk(2, dim=-1)
        topk_prob = topk_prob / topk_prob.sum(-1, keepdim=True)
        return topk_idx, topk_prob

# MoE 层：调度 + 聚合
class MoE(nn.Module):
    def __init__(self, d_model, d_hidden, n_experts):
        super().__init__()
        self.experts = nn.ModuleList([Expert(d_model, d_hidden) for _ in range(n_experts)])
        self.router = Router(d_model, n_experts)

    def forward(self, x):
        # x: [B,T,D]
        B, T, D = x.shape
        idx, prob = self.router(x)   # [B,T,2], [B,T,2]

        y = torch.zeros_like(x)
        for k in range(2):           # top-2 experts
            e_idx = idx[..., k]      # [B,T]
            p = prob[..., k][..., None]  # [B,T,1]
            for e in range(len(self.experts)):
                mask = (e_idx == e)
                if mask.any():
                    y[mask] += p[mask] * self.experts[e](x[mask])
        return y